In [56]:
import os

In [57]:
template_integration_at_account = """
I would love to see ${AT_ACCOUNT} in #web3wallet ❤️‍🔥🙏

#WIP_${ID} ${TITLE}

#byDegensForDegens #wips @sonsofcryptolab
"""

template_default = """
I would love this in #web3wallet ❤️‍🔥🙏

#WIP_${ID} ${TITLE}

#byDegensForDegens #wips @sonsofcryptolab
"""


from enum import Enum


class Category(Enum):
    UNKNOWN = "unknown"
    INFRASTRUCTURE = "infrastructure"
    INTEGRATION = "integration"
    FEATURE = "feature"

    @staticmethod
    def fromId(id: str):
        if id[:1] == "1":
            return Category.INFRASTRUCTURE
        if id[:1] == "2":
            return Category.INTEGRATION
        if id[:1] == "3":
            return Category.FEATURE
        return Category.UNKNOWN


def tweet(id, title, url, atAccount):
    tweet_str = template_default
    if Category.fromId(id) == Category.INTEGRATION and atAccount is not None:
        tweet_str = template_integration_at_account
        tweet_str = tweet_str.replace("${AT_ACCOUNT}", atAccount)

    tweet_str = tweet_str.replace("${ID}", id)
    tweet_str = tweet_str.replace("${TITLE}", title)
    tweet_str = tweet_str.replace("${URL}", url)
    return tweet_str.strip()

In [58]:
from pathlib import Path

# Easiest way to get twitter preview cards changed is new url,
# changing version each time images are updated

curr_version = "v2"
version = "v3"

if curr_version != version:
    for p in Path('.').glob('proposals/*.md'):
        text = p.read_text()
        text = text.replace(curr_version, version)
        with open(p, 'w') as file:
            file.write(text)

In [59]:
# Create proposals json file from ./proposals/*.md files

proposals = []

import re, json

for p in Path('.').glob('proposals/*.md'):
    filename = p.name.replace(".md", "")
    id = filename.replace("wip-", "")
    category = id[:1] # First number in id
    text = p.read_text()
    # WIP-1001 ETH 2.0 support
    title = re.search("\n\#\s+WIP-\d+\s+(.+)\n*", text).group(1)
    body = text[text.index(title) + len(title):].strip()
    atAccount = re.search(".*\[_metadata_:at_account\]:-\s\"*(.+)\s*\"", text)
    atAccount = None if atAccount is None else atAccount.group(1)
    page_url = ("https://sonsofcrypto.com/web3wallet-improvement-proposals/"+version+"/static/"+id+".html")

    proposals.append({
        "id": id,
        "title": title,
        "body": body,
        "category": Category.fromId(id).value,
        "at_account": atAccount,
        "tweet": tweet(id, title, page_url, atAccount),
        "image_url": ("https://sonsofcrypto.com/web3wallet-improvement-proposals/"+version+"/images/"+id+".png"),
        "page_url": page_url,
        "creation_date": "2022-08-28T00:00:00.000Z",
        "votes": 0,
    })
file = open('proposals-list.json', 'w')
file.write(json.dumps(proposals))
file.close()

In [60]:
# Update index.html ul li with proposals

import functools

def html_elem(p: dict) -> str:
    base_url = "https://github.com/sonsofcrypto/web3wallet-improvement-proposals/blob/master/proposals"
    return '<li><a href="'+base_url+'/wip-'+p["id"]+'.md" target="_blank">WIP-'+p["id"]+' '+p["title"]+'</a></li>'

def filer_category(c: Category, p: dict) -> [dict]:
    return p["category"] == c.value

def html_str(proposals: [dict], category: Category) -> str:
    result = filter(lambda p: filer_category(category, p), proposals)
    result = sorted(result, key=lambda p: p["id"])
    result = map(html_elem, result)
    return functools.reduce(lambda r, x: r+"\n"+x, result)

template = Path("template_index.html").read_text()
template = template.replace("${LIST_INFRASTRUCTURE}", html_str(proposals, Category.INFRASTRUCTURE))
template = template.replace("${LIST_INTEGRATION}", html_str(proposals, Category.INTEGRATION))
template = template.replace("${LIST_FEATURES}", html_str(proposals, Category.FEATURE))

with open("index.html", 'w') as file:
    file.write(template)

In [61]:
# Create static page from template each proposal ./static/${PROPOSAL_ID}.html

import urllib

templateFile = open('./' + version + '/static/template.html', 'r')
template = templateFile.read()
templateFile.close()

for p in proposals:
    t = template
    t = t.replace("${ID}", p["id"])
    t = t.replace("${TITLE}", p["title"])
    t = t.replace("${DESCRIPTION}", "web3Wallet improvement proposal " + p["id"])
    t = t.replace("${IMAGE_URL}", p["image_url"].replace("\\", ""))
    t = t.replace("${CONTENT}", p["body"].replace("\\u", "\\n")) # Picks up \u instead of \n
    t = t.replace("${URL}", p["page_url"].replace("\\", ""))
    t = t.replace("${VOTE_TEXT}", urllib.parse.quote(p["tweet"]))
    t = t.replace("${VOTE_URL}", urllib.parse.quote(p["page_url"].replace("\\", "")))
    file = open('./' + version + '/static/'+p["id"]+'.html', 'w')
    file.write(t)
    file.close()

# Create proposals index page NOTE: in progress
with open('./' + version + '/index.md', 'w') as file:
    for p in sorted(proposals, key=lambda p: p["id"]):
        t = "- WIP-" + p["id"] + " " + p["title"] + "\n"
        file.write(t)

In [66]:
# Have to run step one to have proposals in memory
#
# Might be better ways but this is one way to get a Twitter bearer token
# 1. https://oauth-playground.glitch.me/?id=tweetsRecentSearch
# 2. Run a query and authorize with Twitter account
# 3. Click the dots on the right hand side
# 4. Click Include access token
# 5. It should now be visible in the query, just capture the <code>:
# -H "Authorization: Bearer <code>"
# It will expire after 6 hours or something

# Download votes (number of tweets with hashtag for given proposal)
import os, tweepy, time, json
from datetime import datetime, timedelta, date
from dateutil.parser import isoparse
from dotenv import load_dotenv
load_dotenv()


# Get proposals
proposalsFile = open('./proposals-list.json', 'r')
proposals = json.loads(proposalsFile.read())
proposalsFile.close()
if type(proposals) is not list:
    exit ("No proposals found")
# Get votes
try:
    votesFile = open('./proposals-votes.json', 'r')
    votes = json.loads(votesFile.read())
    votesFile.close()
except:
    votes = False
    print(">>> No votes file found, creating a new file.")

proposalVotes = []


api_key = os.environ.get("TWITTER_API", "")
api_key_secret = os.environ.get("TWITTER_API_SECRET", "")
bearer_token = os.environ.get("TWITTER_BEARER_TOKEN", "")


if api_key == "" or api_key_secret == "":
    exit("API keys not set for twitter, please set in env TWITTER_API and TWITTER_API_SECRET")

# Connect to twitter API
max_results = 100
sleep_time = 0.5 # seconds

#auth = tweepy.OAuth2BearerHandler(bearer_token);
#api = tweepy.Client(auth)
api = tweepy.Client(bearer_token=bearer_token)

print ("Sleeping "+ str(sleep_time) + " seconds each query")

def find_votes(_proposal):
    if not votes:
        return 0
    for _vote in votes:
        if _vote['id'] == _proposal['id']:
            return _vote
    return 0

for proposal in proposals:
    # Needs to happen in loop since time is moving while executing
    today = datetime.utcnow()
    week_ago = today - timedelta(days=7) + timedelta(minutes=1)

    vote = find_votes(proposal)
    start_time = week_ago
    count = 0
    previous_count = 0

    if vote != 0 and vote and vote['start_time'] and week_ago < isoparse(vote['start_time']):
        start_time = isoparse(vote['start_time'])
        count = vote['count']
        previous_count = vote['count']
    start_time = start_time.strftime('%Y-%m-%dT%H:%M:%SZ')

    hashtag = "#WIP_"+proposal.get("id") #"#WIP_3001"

    next_token = 0
    while next_token != -1:
        if (next_token == 0):
            response = api.search_recent_tweets(query=hashtag, start_time=start_time, max_results=max_results)
        else:
            response = api.search_recent_tweets(query=hashtag, next_token=next_token, max_results=max_results)

        response_count = int(response.meta['result_count'])
        count = count+response_count
        print ("Processing: hashtag: "+hashtag+
               ", count: "+ str(count) + " (" + str(count-previous_count) + " new)" +
               ", start_time: "+ today.strftime('%Y-%m-%dT%H:%M:%SZ')
               )

        next_token = -1
        if response_count == 100 and response.meta['next_token']:
            next_token = response.meta['next_token']

        time.sleep(sleep_time)

    proposalVotes.append({
        "count": count,
        "hashtag": hashtag,
        "start_time": today.isoformat(),
        "id": proposal["id"]
    })

    if vote == 0:
        proposal['votes'] = 0 # Set the votes
    else:
        proposal['votes'] = vote['count'] # Set the votes

print("Writing file: proposals-votes.json")
file = open('proposals-votes.json', 'w')
file.write(json.dumps(proposalVotes))
file.close()
print("Done.")

print("Updating file with votes: proposals-list.json")
file = open('proposals-list.json', 'w')
file.write(json.dumps(proposals))
file.close()
print("Done.")

Sleeping 0.5 seconds each query
Processing: hashtag: #WIP_2022, count: 0 (0 new), start_time: 2022-10-11T07:42:48Z
Processing: hashtag: #WIP_2016, count: 7 (0 new), start_time: 2022-10-11T07:42:49Z
Processing: hashtag: #WIP_2006, count: 0 (0 new), start_time: 2022-10-11T07:42:50Z
Processing: hashtag: #WIP_2012, count: 6 (0 new), start_time: 2022-10-11T07:42:51Z
Processing: hashtag: #WIP_2026, count: 3 (0 new), start_time: 2022-10-11T07:42:51Z
Processing: hashtag: #WIP_2002, count: 1 (0 new), start_time: 2022-10-11T07:42:52Z
Processing: hashtag: #WIP_2013, count: 6 (0 new), start_time: 2022-10-11T07:42:53Z
Processing: hashtag: #WIP_2027, count: 8 (0 new), start_time: 2022-10-11T07:42:54Z
Processing: hashtag: #WIP_2003, count: 0 (0 new), start_time: 2022-10-11T07:42:55Z
Processing: hashtag: #WIP_2017, count: 3 (0 new), start_time: 2022-10-11T07:42:56Z
Processing: hashtag: #WIP_2007, count: 11 (0 new), start_time: 2022-10-11T07:42:56Z
Processing: hashtag: #WIP_2028, count: 1 (0 new), star